In [2]:
#export
from tensorflow.python.keras.layers import Add, Conv2D, Input, Lambda
from tensorflow.python.keras.models import Model

from exp.nb_common import normalize, denormalize, pixel_shuffle

In [4]:
#export
def bdsr(scale, num_filters = 64, num_res_blocks = 8, res_block_scaling = None):
    x_in = Input(shape=(None, None, 3))
    x = Lambda(normalize)(x_in)

    x = b = Conv2D(num_filters, 3, padding='same')(x)
    for i in range(num_res_blocks):
        b = res_block(b, num_filters, res_block_scaling)
    x = Conv2D(num_filters, 3, padding='same')(b)
    x = Add()([x, b])

    x = upsample(x, scale, num_filters)
    x = Conv2D(3, 3, padding='same')(x)

    x = Lambda(denormalize)(x)
    return Model(x_in, x, name="bdsr")

In [5]:
def res_block(x_in, filters, scaling):
    x1 = Conv2D(filters, [1, 1], padding = 'same', activation = 'relu')(x_in)
    x2_1 = Conv2D(filters, [3, 1], padding = 'same')(x_in)
    x2_2 = Conv2D(x2_1.shape[3], [1, 3], padding = 'same', activation = 'relu')(x2_1)
    y = Add()([x1, x2_2])
    y = Conv2D(y.shape[3], [1, 1])(y)
    if scale:
        y = Lambda(pixel_shuffle(scale=scale))(y)
        return Conv2D(16, [49, 49], padding = 'valid', activation = 'relu')(y)
    return y

In [6]:
def upsample(x, scale, num_filters):
    def upsample_1(x, factor, **kwargs):
        x = Conv2D(num_filters * (factor ** 2), 3, padding = 'same', **kwargs)(x)
        return Lambda(pixel_shuffle(scale = factor))(x)
    
    if scale == 2:
        x = upsample_1(x, 2, name = 'conv2d_1_scale_2')
    elif scale == 3:
        x = upsample_1(x, 3, name = 'conv2d_1_scale_3')
    elif scale == 4:
        x = upsample_1(x, 2, name = 'conv2d_1_scale_2')
        x = upsample_1(x, 2, name = 'conv2d_2_scale_2')
    
    return x